In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [18]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [23]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KANI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KANI\AppData\Roaming\nltk_data...


True

In [83]:
data=pd.read_csv('dataset.csv')


In [44]:
def preprocess_text(text):
    texr=re.sub(r'<.*?>','',text)
    text=text.translate(str.maketrans('','',string.punctuation))
    text=text.lower()
    stop_words=set(stopwords.words('english'))
    text=''.join(word for word in text.split() if word not in stop_words)
    lemm=WordNetLemmatizer()
    text=''.join(lemm.lemmatize(word) for word in text.split())
    return text

In [53]:
data['review']=data['review'].apply(preprocess_text)

In [55]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['sentiment']=le.fit_transform(data['sentiment'])

In [77]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['review'],data['sentiment'],test_size=0.5,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000)
X_train_tfidf=tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)

from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train_tfidf,y_train)

from sklearn.metrics import classification_report,accuracy_score
y_pred=model.predict(X_test_tfidf)
print(f'Accuracy:{accuracy_score(y_test,y_pred)}')
print(classification_report(y_test,y_pred,target_names=le.classes_))

predictions=model.predict(X_test_tfidf)
positive_reviews=(predictions==le.transform(['positive'])).sum()
negative_reviews=(predictions==le.transform(['negative'])).sum()
print(f'Number of postive reviews : {positive_reviews}')
print(f'Number of negative revies : {negative_reviews}')

Accuracy:0.50176
              precision    recall  f1-score   support

    negative       1.00      0.00      0.00     12483
    positive       0.50      1.00      0.67     12517

    accuracy                           0.50     25000
   macro avg       0.75      0.50      0.34     25000
weighted avg       0.75      0.50      0.34     25000

Number of postive reviews : 24973
Number of negative revies : 27
